In [1]:
import os
from bs4 import BeautifulSoup

In [2]:
slozka = "persony_cs"

In [3]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [4]:
seznam_person = seznam_souboru(slozka)
seznam_person[0:10]

['1000-ondrej-masek.html',
 '100019-kino-svetozor.html',
 '100038-ad-wood-sro.html',
 '100039-hypermarket-film-s-r-o.html',
 '100039-hypermarket-film-sro.html',
 '100039-hypermarket-film.html',
 '100040-milos-novotny.html',
 '100076-punk-film-sro.html',
 '100076-punk-film.html',
 '10009-jan-kaspar.html']

In [5]:
len(seznam_person)

37582

In [6]:
def jake_jsou_bio_udaje(slozka):
    bio = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='item-details')
            for item_detail in item_details:
                h3_tags = item_detail.find_all('h3')
                for h3_tag in h3_tags:
                    if h3_tag.get_text() not in bio:
                        bio.append(h3_tag.get_text())
    return(bio)

In [7]:
kolik = 1000

In [8]:
bios = jake_jsou_bio_udaje(slozka)
bios

['Alternativní jména',
 'Datum narození',
 'Místo narození',
 'Datum úmrtí',
 'Místo úmrtí']

In [9]:
import pandas as pd

In [10]:
def id_filmu(url):
    if "cs/film/" in url:
        id_filmu = url.split("/")[-2]
    else:
        id_filmu = 0
    return(id_filmu)

In [11]:
import re

In [12]:
def scrape_person(slozka, soubor):
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
            
    try:
    
        bio_udaje = {}

        jmeno = soup.find("h1").text.strip()

        id = soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]

        bio_udaje["Jméno"] = f"{jmeno} ({id})"
        
        bio_udaje["Id"] = id

        if soup.find(id="description-body-reducer-bio"):
                        
            bio_udaje["Životopis"] = soup.find(id="description-body-reducer-bio").text.strip()
            
        item_details = soup.find_all(class_='item-details')

        for item_detail in item_details:
                        
            if item_detail.find("div"):
                       
                divs = item_detail.find_all("div")

                for div in item_detail:
                    for b in bios:
                        if b in div.text:
                            bio_udaje[b] = div.text.replace(b,"").strip()
                            
            else:
                bio_udaje["Medailon"] = item_detail.text.strip()
        
        if soup.find(id="person-filmography-list-target"):
            bio_udaje["Osoba"] = "fyzická"
        if soup.find(id="company-filmography-list-target"):
            bio_udaje["Osoba"] = "právnická"
        
        sekce = soup.find_all(class_="db-film-credits-section")
        
        for s in sekce:
            if s.find("h2").text.strip() == "Ocenění":
                oceneni = s.find_all(class_="credit-item")
                ceny = []
                for o in oceneni:
                    if o.find('a'):
                        ceny.append([f"""{o.find('a').text} ({id_filmu(o.find('a')['href'])})""", o.text.strip()])
                    else:
                        ceny.append([None, o.text.strip()])
                bio_udaje["Ocenění"] = ceny
        
        bio_udaje = pd.DataFrame.from_dict([bio_udaje])

    except:
        bio_udaje = pd.DataFrame()
    
    return(bio_udaje)

In [13]:
test = scrape_person("persony_cs","7006-jan-jirasek.html")
test

,Jméno,Id,Medailon,Osoba,Ocenění
0,Jan Jirásek (7006),7006,,fyzická,"[[Kuře melancholik (21929), Nominace\n ..."


In [14]:
test = scrape_person("persony_cs","494-karel-roden.html")
test

,Jméno,Id,Medailon,Datum narození,Místo narození,Osoba,Ocenění
0,Karel Roden nejml. (494),494,herec,18.05.1962,"České Budějovice, Československo",fyzická,"[[None, Nominace\n ..."


In [15]:
test = scrape_person("persony_cs","22659-kuhnuv-detsky-pevecky-sbor.html")
test

,Jméno,Id,Alternativní jména,Osoba
0,Kühnův dětský sbor (22659),22659,Kühnův dětský pěvecký sbor,právnická


In [16]:
test = scrape_person("persony_cs","126270-frantisek-kubka.html")
test

,Jméno,Id,Medailon,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Alternativní jména,Osoba,Ocenění
0,František Kubka (126270),126270,"dramatik, spisovatel, básník, novinář, prozaik...",04.03.1894,"Praha, Rakousko-Uhersko",07.01.1969,"Praha, Československo","A. K. Buk (pseudonym), Cumulus (pseudonym), ...",fyzická,"[[None, Vítěz\n ..."


In [17]:
test = scrape_person("persony_cs","2878-jirina-bohdalova.html")
test['Ocenění'].iloc[0]

[[None,
  'Vítěz\n                                    Festival: 55. dětský filmový a televizní festival Oty Hofmana Ostrov nad Ohří\n                    \n                \n                                                            2023\n                                    \n                                                            Ostrov nad Ohří / Česká republika\n                                    \n                                    Zlatá buřinka Oty Hofmana\n                \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\n                                    udělena 9. 10. 2023 v Domě kultury v Ostrově nad Ohří za zvláštní přínos v oblasti dětského filmu'],
 [None,
  'Vítěz\n                                    Festival: 39. festival české filmové a televizní komedie Nové Město nad Metují\n                    \n                \n                                                            2017\n                                    \n                                                        

In [18]:
def scrape_persony(slozka):
    persony = pd.DataFrame()
    count = 0 
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"done: {count} / total: {len(seznam_souboru(slozka)[:kolik])} / {s}")
        h = scrape_person(slozka, s)
        persony = pd.concat([persony, h])
    persony = persony.reset_index(drop=True)
    return(persony)

In [19]:
kolik = None

In [20]:
persony = scrape_persony("persony_cs")

done: 1 / total: 37582 / 1000-ondrej-masek.html
done: 2 / total: 37582 / 100019-kino-svetozor.html
done: 3 / total: 37582 / 100038-ad-wood-sro.html
done: 4 / total: 37582 / 100039-hypermarket-film-s-r-o.html
done: 5 / total: 37582 / 100039-hypermarket-film-sro.html
done: 6 / total: 37582 / 100039-hypermarket-film.html
done: 7 / total: 37582 / 100040-milos-novotny.html
done: 8 / total: 37582 / 100076-punk-film-sro.html
done: 9 / total: 37582 / 100076-punk-film.html
done: 10 / total: 37582 / 10009-jan-kaspar.html
done: 11 / total: 37582 / 100096-cestmir-spurny.html
done: 12 / total: 37582 / 1001-renata-pavlickova.html
done: 13 / total: 37582 / 10010-ales-zabojnik.html
done: 14 / total: 37582 / 100116-tomas-dalecky.html
done: 15 / total: 37582 / 100137-film-factory-sro.html
done: 16 / total: 37582 / 100138-film-factory-cz-sro.html
done: 17 / total: 37582 / 100139-vojtech-matyas-jirovec.html
done: 18 / total: 37582 / 100156-josef-gabriel.html
done: 19 / total: 37582 / 100196-pegasos-film.h

done: 148 / total: 37582 / 100817-studio-pro-upravu-zahranicnich-filmu.html
done: 149 / total: 37582 / 100818-divadlo-variete-brno-jeviste.html
done: 150 / total: 37582 / 100818-divadlo-variete-brno.html
done: 151 / total: 37582 / 100819-hussein-moussa.html
done: 152 / total: 37582 / 10082-richard-bobek.html
done: 153 / total: 37582 / 10082-riso-bobek.html
done: 154 / total: 37582 / 100823-sssssfilm-sro.html
done: 155 / total: 37582 / 100823-sssssfilm.html
done: 156 / total: 37582 / 100826-snake-catcher-sro.html
done: 157 / total: 37582 / 100826-snake-catcher.html
done: 158 / total: 37582 / 10083-iveta-matyasova.html
done: 159 / total: 37582 / 100838-filmovy-ustav.html
done: 160 / total: 37582 / 100839-bionaut-animation.html
done: 161 / total: 37582 / 100839-bionaut-docs.html
done: 162 / total: 37582 / 100839-bionaut-films.html
done: 163 / total: 37582 / 100839-bionaut-s-r-o.html
done: 164 / total: 37582 / 100839-bionaut-sro.html
done: 165 / total: 37582 / 100839-bionaut.html
done: 166

done: 295 / total: 37582 / 101857-vaclav-munzberger.html
done: 296 / total: 37582 / 101858-triglav.html
done: 297 / total: 37582 / 101859-michael-caine.html
done: 298 / total: 37582 / 101860-baf.html
done: 299 / total: 37582 / 101862-ludvik-sojka.html
done: 300 / total: 37582 / 101864-atelier-sascha-viden.html
done: 301 / total: 37582 / 101864-sascha-viden.html
done: 302 / total: 37582 / 10188-jaroslav-pozzi.html
done: 303 / total: 37582 / 101896-aqs-a-s.html
done: 304 / total: 37582 / 101896-aqs-as.html
done: 305 / total: 37582 / 101897-magic-box-slovakia-sro.html
done: 306 / total: 37582 / 1019-petra-pucikova.html
done: 307 / total: 37582 / 101936-aerofilms-s-r-o.html
done: 308 / total: 37582 / 101936-aerofilms-sro.html
done: 309 / total: 37582 / 101936-aerofilms.html
done: 310 / total: 37582 / 101977-atelier-schonbrunn-viden.html
done: 311 / total: 37582 / 101977-schonbrunn-viden.html
done: 312 / total: 37582 / 101996-barrandov-productions-sro.html
done: 313 / total: 37582 / 102036-

done: 443 / total: 37582 / 102667-sovinfilm.html
done: 444 / total: 37582 / 102668-lazne-darkov-as.html
done: 445 / total: 37582 / 102668-lazne-darkov.html
done: 446 / total: 37582 / 102670-kancelar-generalniho-komisare-pro-ceskou-ucast-expo.html
done: 447 / total: 37582 / 102670-kancelar-generalniho-komisare-ucasti-ceske-republiky-na-vseo.html
done: 448 / total: 37582 / 102671-bionaut-works-sro.html
done: 449 / total: 37582 / 102671-bionaut-works.html
done: 450 / total: 37582 / 102673-orchestr-osvobozeneho-divadla.html
done: 451 / total: 37582 / 102674-fresh-lobster-sro.html
done: 452 / total: 37582 / 102674-fresh-lobster.html
done: 453 / total: 37582 / 102678-buchty-a-loutky.html
done: 454 / total: 37582 / 1027-chloe-amp-helmuth.html
done: 455 / total: 37582 / 1027-chloe-helmuth.html
done: 456 / total: 37582 / 102716-safilm-s-r-o.html
done: 457 / total: 37582 / 102717-karel-cajthaml.html
done: 458 / total: 37582 / 102736-magnetix-tonstudio-ag.html
done: 459 / total: 37582 / 102736-ma

done: 582 / total: 37582 / 1035-jana-zemlickova.html
done: 583 / total: 37582 / 10350-jan-palat.html
done: 584 / total: 37582 / 10350-janek-palat.html
done: 585 / total: 37582 / 10351-jana-poctova.html
done: 586 / total: 37582 / 103516-mowshe-sro.html
done: 587 / total: 37582 / 103516-mowshe.html
done: 588 / total: 37582 / 103516-o-mowshe.html
done: 589 / total: 37582 / 103516-studio-mowshe.html
done: 590 / total: 37582 / 103520-marta-klikarova.html
done: 591 / total: 37582 / 103521-anna-musilova.html
done: 592 / total: 37582 / 103558-saatchi-amp-saatchi-s-r-o.html
done: 593 / total: 37582 / 10357-jaroslav-sadilek.html
done: 594 / total: 37582 / 103596-open-field-pictures-s-r-o.html
done: 595 / total: 37582 / 103596-open-field-pictures.html
done: 596 / total: 37582 / 1036-honza-horacek.html
done: 597 / total: 37582 / 103616-bonny-stiftung-fur-die-freiheit.html
done: 598 / total: 37582 / 10362-nikolaj-vasiljevic-gogol.html
done: 599 / total: 37582 / 103621-fauna-a-film-praha.html
done: 

...

done: 36886 / total: 37582 / 95257-margo-films-paris.html
done: 36887 / total: 37582 / 95277-ministerstvo-obrany-ceske-republiky.html
done: 36888 / total: 37582 / 95277-ministerstvo-obrany-cr.html
done: 36889 / total: 37582 / 95297-hotel-adrie-praha.html
done: 36890 / total: 37582 / 95297-hotel-adrie-vaclavske-namesti-78426-jeviste.html
done: 36891 / total: 37582 / 953-josef-hlavac.html
done: 36892 / total: 37582 / 95356-unarclub-os.html
done: 36893 / total: 37582 / 95356-unarfilm.html
done: 36894 / total: 37582 / 95376-hermann-prey.html
done: 36895 / total: 37582 / 95396-klucivespolek-s-r-o.html
done: 36896 / total: 37582 / 95396-klucivespolek.html
done: 36897 / total: 37582 / 954-hedvika-ilkiwova.html
done: 36898 / total: 37582 / 9546-andrea-kunesova.html
done: 36899 / total: 37582 / 95476-sts-s-r-o-tv-markiza.html
done: 36900 / total: 37582 / 9548-karel-skop.html
done: 36901 / total: 37582 / 95499-ceska-pojistovna-a-s.html
done: 36902 / total: 37582 / 955-lubos-ilkiw.html
done: 3690

done: 37027 / total: 37582 / 96688-milos-bobek.html
done: 37028 / total: 37582 / 9669-michael-kocab.html
done: 37029 / total: 37582 / 96690-milos-subrt.html
done: 37030 / total: 37582 / 96691-mtt-multra-trading-team-praha-spol-s-ro.html
done: 37031 / total: 37582 / 96691-mtt-multra-trading-team.html
done: 37032 / total: 37582 / 96691-multra-trading-team-praha.html
done: 37033 / total: 37582 / 96692-riosport-press-praha.html
done: 37034 / total: 37582 / 96692-riosport-press.html
done: 37035 / total: 37582 / 967-zdenek-dubovy.html
done: 37036 / total: 37582 / 9670-dusan-schaffer.html
done: 37037 / total: 37582 / 96702-jiri-vlcek.html
done: 37038 / total: 37582 / 9671-ivan-filus.html
done: 37039 / total: 37582 / 9672-andrej-ludvig.html
done: 37040 / total: 37582 / 9673-martin-minac.html
done: 37041 / total: 37582 / 96737-neue-thalia-film-wien.html
done: 37042 / total: 37582 / 96738-jump-cut-srl.html
done: 37043 / total: 37582 / 96738-jump-cut.html
done: 37044 / total: 37582 / 96739-trenti

done: 37170 / total: 37582 / 9733-petr-peknic.html
done: 37171 / total: 37582 / 9734-nikola-navratil.html
done: 37172 / total: 37582 / 97341-loutkovy-film-praha.html
done: 37173 / total: 37582 / 97341-studio-loutkoveho-filmu-praha.html
done: 37174 / total: 37582 / 97356-otakar-prochazka.html
done: 37175 / total: 37582 / 9738-ondrej-ladek.html
done: 37176 / total: 37582 / 9738-xindl-x.html
done: 37177 / total: 37582 / 9739-miluse-splechtova-hrusinska.html
done: 37178 / total: 37582 / 9739-miluse-splechtova.html
done: 37179 / total: 37582 / 97397-vyrobni-skupina-zdenka-reimanna.html
done: 37180 / total: 37582 / 97399-dor-film.html
done: 37181 / total: 37582 / 974-jiri-kulhavy.html
done: 37182 / total: 37582 / 97400-orf-wien.html
done: 37183 / total: 37582 / 97400-orf.html
done: 37184 / total: 37582 / 97400-osterreichischer-rundfunk.html
done: 37185 / total: 37582 / 97416-kresleny-film-praha.html
done: 37186 / total: 37582 / 97416-studio-kresleneho-filmu-praha.html
done: 37187 / total: 37

done: 37317 / total: 37582 / 98437-bio-illusion.html
done: 37318 / total: 37582 / 98438-ceska-produkcni-invest-a-s.html
done: 37319 / total: 37582 / 9844-stefan-nagy.html
done: 37320 / total: 37582 / 9845-miroslav-rapcan.html
done: 37321 / total: 37582 / 9846-viera-rajtarova.html
done: 37322 / total: 37582 / 9847-maria-chmelova.html
done: 37323 / total: 37582 / 98477-nora-houfova.html
done: 37324 / total: 37582 / 9848-rudolf-grznar.html
done: 37325 / total: 37582 / 98482-axman-production-spol-s-r-o.html
done: 37326 / total: 37582 / 98482-axman-production-spol-s-ro.html
done: 37327 / total: 37582 / 98482-axman-production.html
done: 37328 / total: 37582 / 9849-ladislav-elgyutt.html
done: 37329 / total: 37582 / 98496-biokovofilm-s-r-o.html
done: 37330 / total: 37582 / 98496-biokovofilm.html
done: 37331 / total: 37582 / 9850-peter-gajdos.html
done: 37332 / total: 37582 / 9851-maros-kozich.html
done: 37333 / total: 37582 / 98517-muzeum-jihovychodni-moravy-zlin.html
done: 37334 / total: 3758

done: 37459 / total: 37582 / 99429-zweite-babelsberg-film-gmbh.html
done: 37460 / total: 37582 / 99439-anna-steimarova.html
done: 37461 / total: 37582 / 99440-malirna-a-kulisarna-narodniho-divadla-u-apolinare-na-karlove.html
done: 37462 / total: 37582 / 99440-narodni-divadlo-kulisarna.html
done: 37463 / total: 37582 / 99441-chelsea-winstanley.html
done: 37464 / total: 37582 / 99457-miroslav-innemann.html
done: 37465 / total: 37582 / 99459-miroslav-kahoun.html
done: 37466 / total: 37582 / 9946-jitka-novakova.html
done: 37467 / total: 37582 / 99460-u-f-o-pictures-s-r-o.html
done: 37468 / total: 37582 / 99460-u-f-o-pictures.html
done: 37469 / total: 37582 / 99460-ufo-pictures-sro.html
done: 37470 / total: 37582 / 99460-ufo-pictures.html
done: 37471 / total: 37582 / 9947-filip-blazek.html
done: 37472 / total: 37582 / 99476-magistrat-hl-m-prahy.html
done: 37473 / total: 37582 / 99476-magistrat-hlavniho-mesta-prahy.html
done: 37474 / total: 37582 / 99478-ateliery-viden.html
done: 37475 / tot

In [21]:
persony.sample(15)

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
9529,Eva Melicharová (130070),130070,"pedagožka, tanečnice",fyzická,NaN,"Eva Dumková (provdaná), Eva Gogová (provdaná)",NaN,15.02.1938,"Zábřeh nad Odrou, Československo",25.12.2016,"Praha, Česká republika"
36868,Josef Pokorný (96687),96687,výrobce,fyzická,NaN,"Josef František Pokorný (rodné jméno), Josef F...",NaN,30.03.1869,"Praha, Rakousko-Uhersko",18.06.1917,"Praha, Rakousko-Uhersko"
21801,Karel Bělohoubek (45249),45249,,fyzická,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10395,Lubomír Mindoš (13632),13632,,fyzická,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3264,Producentské centrum Jana Šustera (122076),122076,NaN,právnická,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5588,Sylva Kamenická (126490),126490,,fyzická,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32569,Jiří Liška (84246),84246,,fyzická,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26445,Josef Janíček (664),664,"hudebník, kytarista",fyzická,NaN,NaN,NaN,28.12.1947,"Praha, Československo",NaN,NaN
25864,Karel Heřmánek (6274),6274,herec,fyzická,"[[None, Vítěz\n ...",NaN,NaN,17.10.1947,"Praha, Československo",NaN,NaN
695,Radio Télévision Belge Francophone (104416),104416,NaN,právnická,NaN,"RTBF (zkratka), RTBF Télévision, RTBF Télévi...",NaN,NaN,NaN,NaN,NaN


In [22]:
persony.to_json(os.path.join("data", "persony_raw.json"), orient="records")